# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

In [ ]:
sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

# Pipeline for Croston

In [ ]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = Croston_output_all_params(sales_lumpy,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 4.356670981083334 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_1_005_CA_1_validation,5.0,0.309893,0.1
1,FOODS_1_005_CA_1_validation,3.0,0.401259,0.1
2,FOODS_1_005_CA_1_validation,3.0,0.456317,0.1
3,FOODS_1_005_CA_1_validation,2.0,0.514820,0.1
4,FOODS_1_005_CA_1_validation,11.0,0.551828,0.1
...,...,...,...,...
424903,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9
424904,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9
424905,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9
424906,FOODS_3_827_CA_4_validation,0.0,0.000000,0.9


In [ ]:
pattern_df_result_all_params.to_csv('Croston_Lumpy_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.38558846515000106 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_2_validation,0.1,0.824496,110.6806,123.8461,53.3083,1.465771,1.730638
1,FOODS_1_001_CA_2_validation,0.2,0.839245,112.6605,124.1892,54.8582,1.491990,1.777225
2,FOODS_1_001_CA_2_validation,0.3,0.859357,115.3603,125.1307,57.1069,1.527745,1.827505
3,FOODS_1_001_CA_2_validation,0.4,0.885833,118.9145,126.8805,60.6684,1.574814,1.883550
4,FOODS_1_001_CA_2_validation,0.5,0.910037,122.1636,128.2290,63.4686,1.617843,1.946281
...,...,...,...,...,...,...,...,...
11479,FOODS_3_827_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
11480,FOODS_3_827_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
11481,FOODS_3_827_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
11482,FOODS_3_827_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
243,FOODS_1_017_CA_1_validation,0.1,inf,inf,200.0,NaN,0.335838,0.335838
244,FOODS_1_017_CA_1_validation,0.2,inf,inf,200.0,NaN,0.447389,0.447389
245,FOODS_1_017_CA_1_validation,0.3,inf,inf,200.0,NaN,0.569649,0.569649
246,FOODS_1_017_CA_1_validation,0.4,inf,inf,200.0,NaN,0.701831,0.701831
247,FOODS_1_017_CA_1_validation,0.5,inf,inf,200.0,NaN,0.857411,0.857411
...,...,...,...,...,...,...,...,...
11479,FOODS_3_827_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
11480,FOODS_3_827_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
11481,FOODS_3_827_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
11482,FOODS_3_827_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
243,FOODS_1_017_CA_1_validation,0.1,inf,inf,200.0,NaN,0.335838,0.335838
244,FOODS_1_017_CA_1_validation,0.2,inf,inf,200.0,NaN,0.447389,0.447389
245,FOODS_1_017_CA_1_validation,0.3,inf,inf,200.0,NaN,0.569649,0.569649
246,FOODS_1_017_CA_1_validation,0.4,inf,inf,200.0,NaN,0.701831,0.701831
247,FOODS_1_017_CA_1_validation,0.5,inf,inf,200.0,NaN,0.857411,0.857411
...,...,...,...,...,...,...,...,...
11479,FOODS_3_827_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
11480,FOODS_3_827_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
11481,FOODS_3_827_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
11482,FOODS_3_827_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
216,FOODS_1_015_CA_4_validation,0.1,0.529054,108.75,NaN,100.0,0.029392,0.164489
217,FOODS_1_015_CA_4_validation,0.2,0.529054,108.75,NaN,100.0,0.029392,0.164489
218,FOODS_1_015_CA_4_validation,0.3,0.529054,108.75,NaN,100.0,0.029392,0.164489
219,FOODS_1_015_CA_4_validation,0.4,0.529054,108.75,NaN,100.0,0.029392,0.164489
220,FOODS_1_015_CA_4_validation,0.5,0.529054,108.75,NaN,100.0,0.029392,0.164489
...,...,...,...,...,...,...,...,...
11479,FOODS_3_827_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
11480,FOODS_3_827_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
11481,FOODS_3_827_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
11482,FOODS_3_827_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
243,FOODS_1_017_CA_1_validation,0.1,inf,inf,200.0,NaN,0.335838,0.335838
244,FOODS_1_017_CA_1_validation,0.2,inf,inf,200.0,NaN,0.447389,0.447389
245,FOODS_1_017_CA_1_validation,0.3,inf,inf,200.0,NaN,0.569649,0.569649
246,FOODS_1_017_CA_1_validation,0.4,inf,inf,200.0,NaN,0.701831,0.701831
247,FOODS_1_017_CA_1_validation,0.5,inf,inf,200.0,NaN,0.857411,0.857411
...,...,...,...,...,...,...,...,...
11479,FOODS_3_827_CA_4_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
11480,FOODS_3_827_CA_4_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
11481,FOODS_3_827_CA_4_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
11482,FOODS_3_827_CA_4_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 18.887147335423197 %
Percentage of unexpected values of WMAPE is: 18.887147335423197 %
Percentage of unexpected values of SMAPE is: 20.689655172413794 %
Percentage of unexpected values of MAPE is: 18.887147335423197 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,1.008730,129.867663,123.223196,64.084980,1.827354,2.392612
0.2,0.997073,126.353528,122.940495,65.168419,1.839966,2.411016
0.3,1.009632,126.834816,123.299371,66.869732,1.878243,2.458482
0.4,1.030218,128.741940,123.894922,68.797321,1.921863,2.515576
0.5,1.055198,131.411617,124.629695,70.984764,1.968848,2.579273
0.6,1.084604,134.770003,125.438606,73.350638,2.019803,2.651031
0.7,1.119048,138.865269,126.286766,75.887213,2.075208,2.733535
0.8,1.159907,143.866783,127.194484,78.703716,2.136931,2.830570
0.9,1.209682,150.170150,128.194492,82.003915,2.208404,2.948897


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.2
The optimum Alpha based on Mean WMAPE is: 0.2
The optimum Alpha based on Mean SMAPE is: 0.2
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.1
The optimum Alpha based on Mean RMSE is: 0.1


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.838135,98.0567,117.2054,59.2904,1.402150,1.823928
0.2,0.853963,99.5694,116.4308,60.4327,1.422632,1.835921
0.3,0.867543,100.9506,117.4353,61.9047,1.448913,1.867755
0.4,0.885945,103.3475,118.5589,64.3506,1.483761,1.905419
0.5,0.906675,106.3667,119.5641,65.5883,1.532888,1.953187
0.6,0.925787,109.0156,120.5567,67.8727,1.574597,2.018073
0.7,0.947315,111.6188,121.5057,70.3768,1.627144,2.101925
0.8,0.971281,115.0294,123.0368,72.7419,1.681768,2.164636
0.9,0.996922,117.9201,124.3940,76.1819,1.757485,2.274040


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.1
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.2
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.2

In [ ]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_lumpy, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.17705242246666822 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_2_validation,0.598120,71.9910,89.7073,38.9223,0.874176,1.025028
1,FOODS_1_001_CA_3_validation,1.041195,179.4059,174.3592,49.2134,1.281471,1.921953
2,FOODS_1_002_CA_2_validation,0.667491,119.8060,141.5072,48.0183,0.513454,0.517313
3,FOODS_1_003_CA_2_validation,1.048236,80.2752,93.0550,80.3631,2.580274,3.198129
4,FOODS_1_003_CA_3_validation,0.893063,123.6549,131.4357,30.9872,0.618275,0.703493
...,...,...,...,...,...,...,...
1271,FOODS_3_824_CA_1_validation,1.660014,357.5415,194.9391,78.4736,0.255387,0.294580
1272,FOODS_3_824_CA_2_validation,1.175843,253.2585,196.7091,86.9822,0.180899,0.263125
1273,FOODS_3_827_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
1274,FOODS_3_827_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000


In [ ]:
df_result_final.isnull().sum()

Product      0
MASE       223
WMAPE      223
SMAPE      231
MAPE       303
MAE          0
RMSE         0
dtype: int64

In [ ]:
df_result_final.to_csv('Croston_Lumpy_Test_Data.csv')